## Data Wrangling:
### 1. Web Scraping
### 2. Pandas Basics

## 1. Web Scraping

### 1.1 HTML example

In [1]:
import urllib.request
import numpy as np
from requests import get
from time import sleep
import random

In [2]:
url = 'https://www.wikipedia.org'
with urllib.request.urlopen(url) as response:
    html = response.read()

In [3]:
# Response example, first 1000 characters
html[:1000]

b'<!DOCTYPE html>\n<html lang="en" class="no-js">\n<head>\n<meta charset="utf-8">\n<title>Wikipedia</title>\n<meta name="description" content="Wikipedia is a free online encyclopedia, created and edited by volunteers around the world and hosted by the Wikimedia Foundation.">\n<script>\ndocument.documentElement.className = document.documentElement.className.replace( /(^|\\s)no-js(\\s|$)/, "$1js-enabled$2" );\n</script>\n<meta name="viewport" content="initial-scale=1,user-scalable=yes">\n<link rel="apple-touch-icon" href="/static/apple-touch/wikipedia.png">\n<link rel="shortcut icon" href="/static/favicon/wikipedia.ico">\n<link rel="license" href="//creativecommons.org/licenses/by-sa/3.0/">\n<style>\n.sprite{background-image:linear-gradient(transparent,transparent),url(portal/wikipedia.org/assets/img/sprite-e99844f6.svg);background-repeat:no-repeat;display:inline-block;vertical-align:middle}.svg-Commons-logo_sister{background-position:0 0;width:47px;height:47px}.svg-MediaWiki-logo_sister

### 1.2 IMBD extraction with BeautifulSoup

In [4]:
from bs4 import BeautifulSoup


In [5]:
headers = {'Accept-Language': 'en-US,en;q=0.8'} # If this is not specified, the default language is E  
#get request for sci-fi
response = get("https://www.imdb.com/search/title?genres=sci-fi&start=1&explore=title_type,genres&ref_=adv_prv",
               headers=headers)
    


In [6]:
response = get("https://www.imdb.com/search/title?genres=sci-fi&start=1&explore=title_type,genres&ref_=adv_prv",
         )

In [7]:
page_html = BeautifulSoup(response.text, 'html.parser')

In [8]:
movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

In [9]:
# Response example, first 1000 characters
movie_containers[:1000]

[<div class="lister-item mode-advanced">
 <div class="lister-top-right">
 <div class="ribbonize" data-caller="filmosearch" data-tconst="tt10857160"></div>
 </div>
 <div class="lister-item-image float-left">
 <a href="/title/tt10857160/"> <img alt="She-Hulk: Abogada" class="loadlate" data-tconst="tt10857160" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BMjU4MTkxNzktNzUyYy00NDM2LWE5NGQtNjJlN2Q0N2MxZDAxXkEyXkFqcGdeQXVyMTkxNjUyNQ@@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
 </a> </div>
 <div class="lister-item-content">
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt10857160/">She-Hulk: Abogada</a>
 <span class="lister-item-year text-muted unbold">(2022)</span>
 </h3>
 <p class="text-muted">
 <span class="certificate">16</span>
 <span class="ghost">|</span>
 <span class="runtime">35 min</span>
 <span class="ghost">|</span>
 <span class="gen

In [10]:
titles = [] # for titles appending

In [11]:
for container in movie_containers:

    #conditional for all with metascore
    if container.find('div', class_ = 'ratings-metascore') is not None:

        #title
        title = container.h3.a.text
        titles.append(title)

In [12]:
titles

['The Munsters',
 'Thor: Amor Y Trueno',
 'Avatar',
 '¡Nop!',
 'DC Liga De Súpermascotas',
 'Vesper',
 'Todo en Todas Partes al Mismo Tiempo',
 'Jurassic World Dominio',
 'Rogue One: Una historia de Star Wars',
 'Depredador: La Presa',
 'Moonfall',
 'Minions: Nace Un Villano',
 'Spider-Man: Sin Camino A Casa',
 'Duna',
 'Némesis',
 'Meet Cute',
 'Doctor Strange En El Multiverso De La Locura']

## 2. Pandas Basics


### 2.1  Series

In [13]:
import pandas as pd
import numpy as np

#### A Series is a one-dimensional array-like object containing a sequence of values (of similar types to NumPy types) of the same type and an associated array of data labels, called its index.

In [14]:
obj = pd.Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [15]:
obj.array

<PandasArray>
[4, 7, -5, 3]
Length: 4, dtype: int64

Since we did not specify an index for the data, a default one consisting of the integers 0 through N - 1

In [16]:
obj.index

RangeIndex(start=0, stop=4, step=1)

In [17]:
obj2 = pd.Series([4, 7, -5, 3], index=["d", "b", "a", "c"])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [18]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

You can use labels in the index when selecting single values or a set of values:

In [19]:
obj2["a"]

-5

In [20]:
obj2["d"] = 6
obj2[["c", "a", "d"]]

c    3
a   -5
d    6
dtype: int64

Using NumPy functions or NumPy-like operations, such as filtering with a Boolean array, scalar multiplication, or applying math functions, will preserve the index-value link:

In [21]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [22]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [23]:
import numpy as np
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

Another way to think about a Series is as a fixed-length, ordered dictionary, as it is a mapping of index values to data values. It can be used in many contexts where you might use a dictionary:

In [24]:
"e" in obj2

False

In [25]:
"b" in obj2

True

You can create a Series from a dictionary:



In [26]:
sdata = {"Ohio": 35000, "Texas": 71000, "Oregon": 16000, "Utah": 5000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [27]:
obj3.to_dict()

{'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}

When you are only passing a dictionary, the index in the resulting Series will respect the order of the keys according to the dictionary’s keys method, which depends on the key insertion order. You can override this by passing an index with the dictionary keys in the order you want them to appear in the resulting Series:

In [28]:
states = ["California", "Ohio", "Oregon", "Texas"]
obj4 = pd.Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

 The isna and notna functions in pandas should be used to detect missing data:

In [29]:
pd.isna(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [30]:
pd.notna(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [31]:
obj4.isna()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

A useful Series feature for many applications is that it automatically aligns by index label in arithmetic operations:

In [32]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [33]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [34]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Both the Series object itself and its index have a name attribute, which integrates with other areas of pandas functionality:

In [35]:
obj4.name = "population"
obj4.index.name = "state"
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

In [36]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [37]:
obj.index = ["Bob", "Steve", "Jeff", "Ryan"]
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

### 2.2 DataFrames

#### A DataFrame represents a rectangular table of data and contains an ordered, named collection of columns, each of which can be a different value type (numeric, string, Boolean, etc.). The DataFrame has both a row and column index; it can be thought of as a dictionary of Series all sharing the same index.

There are many ways to construct a DataFrame, though one of the most common is from a dictionary of equal-length lists or NumPy arrays:

In [38]:
data = {"state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
        "year": [2000, 2001, 2002, 2001, 2002, 2003],
        "pop": [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
df = pd.DataFrame(data)

In [39]:
df

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


You can use `head` and `tail` for selection of first and last rows:

In [40]:
df.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


In [41]:
df.tail()

,state,year,pop
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


If you specify a sequence of columns, the DataFrame’s columns will be arranged in that order. Notice the missing values in 'debt':

In [42]:
df2 = pd.DataFrame(data, columns=["year", "state", "pop", "debt"])
df2

,year,state,pop,debt
0,2000,Ohio,1.5,NaN
1,2001,Ohio,1.7,NaN
2,2002,Ohio,3.6,NaN
3,2001,Nevada,2.4,NaN
4,2002,Nevada,2.9,NaN
5,2003,Nevada,3.2,NaN


In [43]:
df2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

A column in a DataFrame can be retrieved as a Series either by dictionary-like notation or by using the dot attribute notation:



In [44]:
df2["state"]

0      Ohio
1      Ohio
2      Ohio
3    Nevada
4    Nevada
5    Nevada
Name: state, dtype: object

In [45]:
df2.year

0    2000
1    2001
2    2002
3    2001
4    2002
5    2003
Name: year, dtype: int64

Rows can also be retrieved by position or name with the special iloc and loc attributes (more on this later in “Selection on DataFrame with loc and iloc”):

In [46]:
df2.loc[1]

year     2001
state    Ohio
pop       1.7
debt      NaN
Name: 1, dtype: object

In [47]:
df2.iloc[2]

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: 2, dtype: object

Columns can be modified by assignment. For example, the empty debt column could be assigned a scalar value or an array of values:

In [48]:
df2["debt"] = 16.5
df2

,year,state,pop,debt
0,2000,Ohio,1.5,16.5
1,2001,Ohio,1.7,16.5
2,2002,Ohio,3.6,16.5
3,2001,Nevada,2.4,16.5
4,2002,Nevada,2.9,16.5
5,2003,Nevada,3.2,16.5


In [49]:
df2["debt"] = np.arange(6.)
df2

,year,state,pop,debt
0,2000,Ohio,1.5,0.0
1,2001,Ohio,1.7,1.0
2,2002,Ohio,3.6,2.0
3,2001,Nevada,2.4,3.0
4,2002,Nevada,2.9,4.0
5,2003,Nevada,3.2,5.0


When you are assigning lists or arrays to a column, the value’s length must match the length of the DataFrame. If you assign a Series, its labels will be realigned exactly to the DataFrame’s index, inserting missing values in any index values not present:

In [50]:
val = pd.Series([-1.2, -1.5, -1.7], index=["two", "four", "five"])
df2["debt"] = val
df2

,year,state,pop,debt
0,2000,Ohio,1.5,NaN
1,2001,Ohio,1.7,NaN
2,2002,Ohio,3.6,NaN
3,2001,Nevada,2.4,NaN
4,2002,Nevada,2.9,NaN
5,2003,Nevada,3.2,NaN


Assigning a column that doesn’t exist will create a new column.

The del keyword will delete columns like with a dictionary. As an example, I first add a new column of Boolean values where the state column equals "Ohio":

In [51]:
df2["eastern"] = df2["state"] == "Ohio"
df2

,year,state,pop,debt,eastern
0,2000,Ohio,1.5,NaN,True
1,2001,Ohio,1.7,NaN,True
2,2002,Ohio,3.6,NaN,True
3,2001,Nevada,2.4,NaN,False
4,2002,Nevada,2.9,NaN,False
5,2003,Nevada,3.2,NaN,False


The del method can then be used to remove this column:

In [52]:
del df2["eastern"]
df2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

Another common form of data is a nested dictionary of dictionaries:



In [53]:
populations = {"Ohio": {2000: 1.5, 2001: 1.7, 2002: 3.6},
               "Nevada": {2001: 2.4, 2002: 2.9}}

If the nested dictionary is passed to the DataFrame, pandas will interpret the outer dictionary keys as the columns, and the inner keys as the row indices:

In [54]:
df3 = pd.DataFrame(populations)
df3

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


You can transpose the DataFrame (swap rows and columns) with similar syntax to a NumPy array:



In [55]:
df3.T

,2000,2001,2002
Ohio,1.5,1.7,3.6
Nevada,NaN,2.4,2.9


If a DataFrame’s index and columns have their name attributes set, these will also be displayed:

In [56]:
df3

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


In [57]:
df3.index.name = "year"
df3.columns.name = "state"
df3

state,Ohio,Nevada
year,,
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


DataFrame’s to_numpy method returns the data contained in the DataFrame as a two-dimensional ndarray:

In [58]:
df3.to_numpy()

array([[1.5, nan],
       [1.7, 2.4],
       [3.6, 2.9]])

In [59]:
df2.to_numpy()

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, nan],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, nan],
       [2002, 'Nevada', 2.9, nan],
       [2003, 'Nevada', 3.2, nan]], dtype=object)

#### Reindexing
Reindex means to create a new object with the values rearranged to align with the new index:



In [60]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=["d", "b", "a", "c"])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [61]:
obj2 = obj.reindex(["a", "b", "c", "d", "e"])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

For ordered data like time series, you may want to do some interpolation or filling of values when reindexing. The method option allows us to do this, using a method such as `ffill`, which forward-fills the values:



In [62]:
obj3 = pd.Series(["blue", "purple", "yellow"], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [63]:
obj3.reindex(np.arange(6), method="ffill")

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

With DataFrame, reindex can alter the (row) index, columns, or both. When passed only a sequence, it reindexes the rows in the result:

In [64]:
df = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=["a", "c", "d"],
                     columns=["Ohio", "Texas", "California"])
df

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [65]:
df2 = df.reindex(index=["a", "b", "c", "d"])
df2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


The columns can be reindexed with the columns keyword:



In [66]:
states = ["Texas", "Utah", "California"]
df.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


#### Dropping entries
Dropping one or more entries from an axis is simple if you already have an index array or list without those entries, since you can use the reindex method or .loc-based indexing.

In [67]:
obj = pd.Series(np.arange(5.), index=["a", "b", "c", "d", "e"])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [68]:
new_obj = obj.drop("c")
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [69]:
obj.drop(["d", "c"])

a    0.0
b    1.0
e    4.0
dtype: float64

In [70]:
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

With DataFrame, index values can be deleted from either axis. To illustrate this, we first create an example DataFrame:

In [71]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=["Ohio", "Colorado", "Utah", "New York"],
                    columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


Calling drop with a sequence of labels will drop values from the row labels (axis 0):



In [72]:
data.drop(index=["Colorado", "Ohio"])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


To drop labels from the columns, instead use the columns keyword:

In [73]:
data.drop(columns=["two"])

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


You can also drop values from the columns by passing axis=1 (which is like NumPy) or axis="columns":

In [74]:
data.drop("two", axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [75]:
data.drop(["two", "four"], axis="columns")

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


#### Indexing, Selection, and Filtering
Series indexing works analogously to NumPy array indexing, except you can use the Series’s index values instead of only integers:

In [76]:
obj = pd.Series(np.arange(4.), index=["a", "b", "c", "d"])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [77]:
obj["b"]

1.0

In [78]:
obj[1]

1.0

In [79]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [80]:
obj[["b", "a", "d"]]

b    1.0
a    0.0
d    3.0
dtype: float64

In [81]:
obj[[1, 3]]

b    1.0
d    3.0
dtype: float64

In [82]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

While you can select data by label this way, the preferred way to select index values is with the special loc operator:



In [83]:
obj.loc[["b", "a", "d"]]

b    1.0
a    0.0
d    3.0
dtype: float64

The reason to prefer loc is because of the different treatment of integers when indexing with `[]`. Regular `[]`-based indexing will treat integers as labels if the index contains integers, so the behavior differs depending on the data type of the index. For example:

In [84]:
obj1 = pd.Series([1, 2, 3], index=[2, 0, 1])
obj2 = pd.Series([1, 2, 3], index=["a", "b", "c"])

In [85]:
obj1

2    1
0    2
1    3
dtype: int64

In [86]:
obj2

a    1
b    2
c    3
dtype: int64

In [87]:
obj1[[0, 1, 2]]

0    2
1    3
2    1
dtype: int64

In [88]:
obj2[[0, 1, 2]]

a    1
b    2
c    3
dtype: int64

When using `loc`, the expression `obj.loc[[0, 1, 2]]` will fail when the index does not contain integers:

In [89]:
obj2.loc[[0, 1]]

KeyError: "None of [Int64Index([0, 1], dtype='int64')] are in the [index]"

Since loc operator indexes exclusively with labels, there is also an iloc operator that indexes exclusively with integers to work consistently whether or not the index contains integers:

In [90]:
obj1.iloc[[0, 1, 2]]


2    1
0    2
1    3
dtype: int64

In [91]:
obj2.iloc[[0, 1, 2]]


a    1
b    2
c    3
dtype: int64

#### Selection on DataFrame with loc and iloc
Like Series, DataFrame has special attributes `loc` and `iloc` for label-based and integer-based indexing, respectively. Since DataFrame is two-dimensional, you can select a subset of the rows and columns with NumPy-like notation using either axis labels `(loc)` or integers `(iloc)`.

Let’s select a single row by label:

In [92]:
data.loc["Colorado"]

one      4
two      5
three    6
four     7
Name: Colorado, dtype: int64

The result of selecting a single row is a Series with an index that contains the DataFrame’s column labels. To select multiple roles, creating a new DataFrame, pass a sequence of labels:

In [93]:
data.loc[["Colorado", "New York"]]

,one,two,three,four
Colorado,4,5,6,7
New York,12,13,14,15


You can combine both row and column selection in loc by separating the selections with a comma:

In [94]:
data.loc["Colorado", ["two", "three"]]

two      5
three    6
Name: Colorado, dtype: int64

We’ll then perform some similar selections with integers using iloc:

In [95]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int64

In [96]:
data.iloc[[2, 1]]

,one,two,three,four
Utah,8,9,10,11
Colorado,4,5,6,7


In [97]:
data.iloc[2, [3, 0, 1]]

four    11
one      8
two      9
Name: Utah, dtype: int64

In [98]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,4,5
Utah,11,8,9


Both indexing functions work with slices in addition to single labels or lists of labels:

In [99]:
data.loc[:"Utah", "two"]

Ohio        1
Colorado    5
Utah        9
Name: two, dtype: int64

In [100]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,4,5,6
Utah,8,9,10
New York,12,13,14


Boolean arrays can be used with loc but not iloc:

In [101]:
data.loc[data.three >= 2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


### 2.3 Combining and Merging Datasets

Data contained in pandas objects can be combined in a number of ways:

`pandas.merge`
Connect rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database `join` operations.

`pandas.concat`
Concatenate or “stack” objects together along an axis.

`combine_first`
Splice together overlapping data to fill in missing values in one object with values from another.


#### DataFrame joins
Merge or join operations combine datasets by linking rows using one or more keys. These operations are particularly important in relational databases (e.g., SQL-based).

In [102]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

In [103]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [104]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [105]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Note that I didn’t specify which column to join on. If that information is not specified, pandas.merge uses the overlapping column names as the keys. It’s a good practice to specify explicitly, though:

In [106]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In general, the order of column output in pandas.merge operations is unspecified. If the column names are different in each object, you can specify them separately:

In [107]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

In [108]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [109]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [110]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


The "c" and "d" values and associated data are missing from the result. By default, pandas.merge does an "inner" join; the keys in the result are the intersection, or the common set found in both tables. Other possible options are "left", "right", and "outer". The outer join takes the union of the keys, combining the effect of applying both left and right joins:

In [111]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0
6,c,3,<NA>
7,d,<NA>,2


In [112]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In an outer join, rows from the left or right DataFrame objects that do not match on keys in the other DataFrame will appear with NA values in the other DataFrame’s columns for the nonmatching rows.



Many-to-many left and inner joins examples:



In [113]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})

In [114]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


## 

In [115]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [116]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [117]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In some cases, the merge key(s) in a DataFrame will be found in its index (row labels). In this case, you can pass `left_index=True` or `right_index=True` (or both) to indicate that the index should be used as the merge key:

In [118]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])


In [119]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [120]:
right1

,group_val
a,3.5
b,7.0


In [121]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


#### Concatenating

Another kind of data combination operation is referred to interchangeably as concatenation or stacking. NumPy’s concatenate function can do this with NumPy arrays:

In [122]:
arr = np.arange(12).reshape((3, 4))


In [123]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [124]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

The concat function in pandas provides a consistent way to combine Series or DataFrames taking into account index information. Suppose we have three Series with no index overlap:



In [125]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [126]:
s1

a    0
b    1
dtype: Int64

In [127]:
s2

c    2
d    3
e    4
dtype: Int64

In [128]:
s3

f    5
g    6
dtype: Int64

In [129]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [130]:
pd.concat([s1, s2, s3], axis="columns")


,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


By default, pandas.concat works along `axis="index"`, producing another Series. If you pass `axis="columns"`, the result will instead be a DataFrame:

In [131]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In this case there is no overlap on the other axis, which as you can see is the union (the "outer" join) of the indexes. You can instead intersect them by passing `join="inner"`:

In [132]:
s4 = pd.concat([s1, s3])

In [133]:
s4

a    0
b    1
f    5
g    6
dtype: Int64

In [134]:
pd.concat([s1, s4], axis="columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [135]:
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


## 3. Extra Reading

### 3.1 Web Scraping with Selenium
#### Suggested Reading: https://www.scrapingbee.com/blog/selenium-python/

### 3.2 Read files to DataFrames
#### Suggested Reading: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

### 3.3 Reshape and Pivoting
#### Suggested Reading: https://pandas.pydata.org/docs/user_guide/reshaping.html

### References:
* https://www.freecodecamp.org/news/web-scraping-sci-fi-movies-from-imdb-with-python/
* https://wesmckinney.com/book/